# Web Scrapping with Beautiful Soup

In [70]:
import requests as r
import pandas as pd
from bs4 import BeautifulSoup
import os

In [78]:
html = r.get(url)
html

<Response [200]>

In [104]:
folder = 'top_100_universities_in_ng'
if not os.path.exists(folder):
    os.makedirs(folder)
with open(os.path.join(folder, "./webometrics_ranking_2019.html"), mode='wb') as file:
    file.write(html.content)

In [105]:
with open(os.path.join(folder, "./webometrics_ranking_2019.html")) as file:
    soup = BeautifulSoup(file, 'lxml')

soup

<!DOCTYPE html>
<html lang="en-US" prefix="og: https://ogp.me/ns#">
<head>
<meta charset="utf-8"/>
<meta content="width=device-width, initial-scale=1" name="viewport"/>
<link href="http://gmpg.org/xfn/11" rel="profile"/>
<!-- Manifest added by SuperPWA - Progressive Web Apps Plugin For WordPress -->
<link href="/superpwa-manifest.json" rel="manifest"/>
<link href="/superpwa-manifest.json" rel="prefetch"/>
<meta content="#D5E0EB" name="theme-color"/>
<meta content="yes" name="mobile-web-app-capable"/>
<meta content="yes" name="apple-touch-fullscreen"/>
<meta content="The Abusites" name="apple-mobile-web-app-title"/>
<meta content="The Abusites" name="application-name"/>
<meta content="yes" name="apple-mobile-web-app-capable"/>
<meta content="default" name="apple-mobile-web-app-status-bar-style"/>
<link href="//i0.wp.com/www.theabusites.com/wp-content/uploads/2020/09/The-Abusites.jpg" rel="apple-touch-icon" sizes="192x192"/>
<link href="//i0.wp.com/www.theabusites.com/wp-content/uploads/

In [107]:
table = soup.find('table')
table

<table class=""><thead><tr><th>ranking</th><th><a href="https://www.webometrics.info/en/Africa/Nigeria?sort=desc&amp;order=World%20Rank" rel="noopener" target="_blank">World Rank</a></th><th><a href="https://www.webometrics.info/en/Africa/Nigeria?sort=asc&amp;order=University" rel="noopener" target="_blank">University</a></th><th><a href="https://www.webometrics.info/en/Africa/Nigeria?sort=asc&amp;order=Det." rel="noopener" target="_blank">Det.</a></th><th><a href="https://www.webometrics.info/en/Africa/Nigeria?sort=asc&amp;order=Presence%20Rank%2A" rel="noopener" target="_blank">Presence Rank*</a></th><th><a href="https://www.webometrics.info/en/Africa/Nigeria?sort=asc&amp;order=Impact%20Rank%2A" rel="noopener" target="_blank">Impact Rank*</a></th><th><a href="https://www.webometrics.info/en/Africa/Nigeria?sort=asc&amp;order=Openness%20Rank%2A" rel="noopener" target="_blank">Openness Rank*</a></th><th><a href="https://www.webometrics.info/en/Africa/Nigeria?sort=asc&amp;order=Excellenc

In [108]:
columns_name = []

for col_name in table.find_all('th'):
    columns_name.append(col_name.text)

columns_name

['ranking',
 'World Rank',
 'University',
 'Det.',
 'Presence Rank*',
 'Impact Rank*',
 'Openness Rank*',
 'Excellence Rank*']

In [148]:
rank = []
wrld_rnk = []
for row in table.find_all('tr'):
    col = row.find_all('td')
    print(col)

[]
[<td>1</td>, <td>1322</td>, <td><a href="https://www.ui.edu.ng/" rel="noreferrer noopener" target="_blank">University of Ibadan</a></td>, <td><a href="https://www.webometrics.info/en/detalles/ui.edu.ng" rel="noopener" target="_blank"></a></td>, <td>2113</td>, <td>2088</td>, <td>1057</td>, <td>1561</td>]
[<td>2</td>, <td>1742</td>, <td><a href="http://covenantuniversity.edu.ng/" rel="noreferrer noopener" target="_blank">Covenant University Ota</a></td>, <td><a href="https://www.webometrics.info/en/detalles/covenantuniversity.edu.ng" rel="noopener" target="_blank"></a></td>, <td>1169</td>, <td>3884</td>, <td>1356</td>, <td>1797</td>]
[<td>3</td>, <td>1805</td>, <td><a href="http://www.unn.edu.ng/" rel="noreferrer noopener" target="_blank">University of Nigeria</a></td>, <td><a href="https://www.webometrics.info/en/detalles/unn.edu.ng" rel="noopener" target="_blank"></a></td>, <td>1311</td>, <td>3279</td>, <td>1038</td>, <td>2243</td>]
[<td>4</td>, <td>1984</td>, <td><a href="https://u

In [147]:
rank = []
wrld_rnk = []
for row in table.find_all('tr'):
    col = row.find_all('td')
    for i in col:
        print(i)

<td>1</td>
<td>1322</td>
<td><a href="https://www.ui.edu.ng/" rel="noreferrer noopener" target="_blank">University of Ibadan</a></td>
<td><a href="https://www.webometrics.info/en/detalles/ui.edu.ng" rel="noopener" target="_blank"></a></td>
<td>2113</td>
<td>2088</td>
<td>1057</td>
<td>1561</td>
<td>2</td>
<td>1742</td>
<td><a href="http://covenantuniversity.edu.ng/" rel="noreferrer noopener" target="_blank">Covenant University Ota</a></td>
<td><a href="https://www.webometrics.info/en/detalles/covenantuniversity.edu.ng" rel="noopener" target="_blank"></a></td>
<td>1169</td>
<td>3884</td>
<td>1356</td>
<td>1797</td>
<td>3</td>
<td>1805</td>
<td><a href="http://www.unn.edu.ng/" rel="noreferrer noopener" target="_blank">University of Nigeria</a></td>
<td><a href="https://www.webometrics.info/en/detalles/unn.edu.ng" rel="noopener" target="_blank"></a></td>
<td>1311</td>
<td>3279</td>
<td>1038</td>
<td>2243</td>
<td>4</td>
<td>1984</td>
<td><a href="https://unilag.edu.ng/" rel="noreferrer no

<td><a href="http://www.esut.edu.ng/" rel="noreferrer noopener" target="_blank">Enugu State University of Science &amp; Technology</a></td>
<td><a href="https://www.webometrics.info/en/detalles/esut.edu.ng" rel="noopener" target="_blank"></a></td>
<td>15624</td>
<td>10334</td>
<td>5306</td>
<td>6084</td>
<td>55</td>
<td>8818</td>
<td><a href="http://fukashere.edu.ng/" rel="noreferrer noopener" target="_blank">Federal University Kashere Gombe State</a></td>
<td><a href="https://www.webometrics.info/en/detalles/fukashere.edu.ng" rel="noopener" target="_blank"></a></td>
<td>5138</td>
<td>11014</td>
<td>5517</td>
<td>6084</td>
<td>56</td>
<td>8935</td>
<td><a href="http://jabu.edu.ng/" rel="noreferrer noopener" target="_blank">Joseph Ayo Babalola University</a></td>
<td><a href="https://www.webometrics.info/en/detalles/jabu.edu.ng" rel="noopener" target="_blank"></a></td>
<td>25775</td>
<td>13131</td>
<td>6690</td>
<td>5178</td>
<td>57</td>
<td>9299</td>
<td><a href="http://fud.edu.ng/" re

<td><a href="http://www.tsuniversity.edu.ng/" rel="noreferrer noopener" target="_blank">Taraba State University Jalingo</a></td>
<td><a href="https://www.webometrics.info/en/detalles/tsuniversity.edu.ng" rel="noopener" target="_blank"></a></td>
<td>18418</td>
<td>14883</td>
<td>6690</td>
<td>6084</td>
<td>98</td>
<td>14110</td>
<td><a href="http://www.osustech.edu.ng/" rel="noreferrer noopener" target="_blank">Ondo State University of Science &amp; Technology Okitipupa</a></td>
<td><a href="https://www.webometrics.info/en/detalles/osustech.edu.ng" rel="noopener" target="_blank"></a></td>
<td>23528</td>
<td>12868</td>
<td>7168</td>
<td>6084</td>
<td>99</td>
<td>14347</td>
<td><a href="https://aul.edu.ng/" rel="noreferrer noopener" target="_blank">Anchor University Lagos</a></td>
<td><a href="https://www.webometrics.info/en/detalles/aul.edu.ng" rel="noopener" target="_blank"></a></td>
<td>9304</td>
<td>17841</td>
<td>5779</td>
<td>6084</td>
<td>100</td>
<td>14446</td>
<td><a href="http:/

In [174]:
rows = table.find_all('tr')
sub = rows[1:]
sub

[<tr><td>1</td><td>1322</td><td><a href="https://www.ui.edu.ng/" rel="noreferrer noopener" target="_blank">University of Ibadan</a></td><td><a href="https://www.webometrics.info/en/detalles/ui.edu.ng" rel="noopener" target="_blank"></a></td><td>2113</td><td>2088</td><td>1057</td><td>1561</td></tr>,
 <tr><td>2</td><td>1742</td><td><a href="http://covenantuniversity.edu.ng/" rel="noreferrer noopener" target="_blank">Covenant University Ota</a></td><td><a href="https://www.webometrics.info/en/detalles/covenantuniversity.edu.ng" rel="noopener" target="_blank"></a></td><td>1169</td><td>3884</td><td>1356</td><td>1797</td></tr>,
 <tr><td>3</td><td>1805</td><td><a href="http://www.unn.edu.ng/" rel="noreferrer noopener" target="_blank">University of Nigeria</a></td><td><a href="https://www.webometrics.info/en/detalles/unn.edu.ng" rel="noopener" target="_blank"></a></td><td>1311</td><td>3279</td><td>1038</td><td>2243</td></tr>,
 <tr><td>4</td><td>1984</td><td><a href="https://unilag.edu.ng/" rel

In [178]:
rank = []
wrld_rnk = []
for i in range(len(sub)):
    rnk = sub[i].find_all('td')[0].text
    wld_rnk = sub[i].find_all('td')[1].text
    
    rank.append(rnk)
    wrld_rnk.append(wld_rnk)



['1', '2', '3', '4', '5', '6', '7', '8', '9', '10', '11', '12', '13', '14', '15', '16', '17', '18', '19', '20', '21', '22', '23', '24', '25', '26', '27', '28', '29', '30', '31', '32', '33', '34', '35', '36', '37', '38', '39', '40', '41', '42', '43', '44', '45', '46', '47', '48', '49', '50', '51', '52', '53', '54', '55', '56', '57', '58', '59', '60', '61', '62', '63', '64', '65', '66', '67', '68', '69', '70', '71', '72', '73', '74', '75', '76', '77', '78', '79', '80', '81', '82', '83', '84', '85', '86', '87', '88', '89', '90', '91', '92', '93', '94', '95', '96', '97', '98', '99', '100']
['1322', '1742', '1805', '1984', '2053', '2378', '2588', '2705', '2819', '3126', '3130', '3157', '3388', '3452', '3510', '3520', '3624', '3664', '3701', '3718', '3803', '3871', '4020', '4077', '4132', '4200', '4514', '5001', '5400', '5484', '5661', '5664', '5700', '5754', '5825', '6351', '6372', '6398', '6471', '6480', '7105', '7340', '7374', '7415', '7482', '7987', '8037', '8066', '8208', '8378', '8418'

In [170]:
rows = table.find_all('tr')
rows[1].find_all('td')[0].text

'1'

In [166]:
rows = table.find_all('tr')
for row in rows:
    col = row.find_all('td')
    col[0]

IndexError: list index out of range

In [54]:
rows = table.find_all('tr')
rows

[<tr><th>ranking</th><th><a href="https://www.webometrics.info/en/Africa/Nigeria?sort=desc&amp;order=World%20Rank" rel="noopener" target="_blank">World Rank</a></th><th><a href="https://www.webometrics.info/en/Africa/Nigeria?sort=asc&amp;order=University" rel="noopener" target="_blank">University</a></th><th><a href="https://www.webometrics.info/en/Africa/Nigeria?sort=asc&amp;order=Det." rel="noopener" target="_blank">Det.</a></th><th><a href="https://www.webometrics.info/en/Africa/Nigeria?sort=asc&amp;order=Presence%20Rank%2A" rel="noopener" target="_blank">Presence Rank*</a></th><th><a href="https://www.webometrics.info/en/Africa/Nigeria?sort=asc&amp;order=Impact%20Rank%2A" rel="noopener" target="_blank">Impact Rank*</a></th><th><a href="https://www.webometrics.info/en/Africa/Nigeria?sort=asc&amp;order=Openness%20Rank%2A" rel="noopener" target="_blank">Openness Rank*</a></th><th><a href="https://www.webometrics.info/en/Africa/Nigeria?sort=asc&amp;order=Excellence%20Rank%2A" rel="noop

In [58]:
for row in table.find_all('tr'):
    c = row.find_all('td')
c

[<td>100</td>,
 <td>14446</td>,
 <td><a href="http://www.fubk.edu.ng/" rel="noreferrer noopener" target="_blank">Federal University Birnin Kebbi</a></td>,
 <td><a href="https://www.webometrics.info/en/detalles/fubk.edu.ng" rel="noopener" target="_blank"></a></td>,
 <td>26140</td>,
 <td>15000</td>,
 <td>6456</td>,
 <td>6084</td>]

In [60]:
rows = table.find_all('th')
rows

[<th>ranking</th>,
 <th><a href="https://www.webometrics.info/en/Africa/Nigeria?sort=desc&amp;order=World%20Rank" rel="noopener" target="_blank">World Rank</a></th>,
 <th><a href="https://www.webometrics.info/en/Africa/Nigeria?sort=asc&amp;order=University" rel="noopener" target="_blank">University</a></th>,
 <th><a href="https://www.webometrics.info/en/Africa/Nigeria?sort=asc&amp;order=Det." rel="noopener" target="_blank">Det.</a></th>,
 <th><a href="https://www.webometrics.info/en/Africa/Nigeria?sort=asc&amp;order=Presence%20Rank%2A" rel="noopener" target="_blank">Presence Rank*</a></th>,
 <th><a href="https://www.webometrics.info/en/Africa/Nigeria?sort=asc&amp;order=Impact%20Rank%2A" rel="noopener" target="_blank">Impact Rank*</a></th>,
 <th><a href="https://www.webometrics.info/en/Africa/Nigeria?sort=asc&amp;order=Openness%20Rank%2A" rel="noopener" target="_blank">Openness Rank*</a></th>,
 <th><a href="https://www.webometrics.info/en/Africa/Nigeria?sort=asc&amp;order=Excellence%20R

In [18]:
rows = table.find_all('td')
rows

[<td>1</td>,
 <td>1322</td>,
 <td><a href="https://www.ui.edu.ng/" rel="noreferrer noopener" target="_blank">University of Ibadan</a></td>,
 <td><a href="https://www.webometrics.info/en/detalles/ui.edu.ng" rel="noopener" target="_blank"></a></td>,
 <td>2113</td>,
 <td>2088</td>,
 <td>1057</td>,
 <td>1561</td>,
 <td>2</td>,
 <td>1742</td>,
 <td><a href="http://covenantuniversity.edu.ng/" rel="noreferrer noopener" target="_blank">Covenant University Ota</a></td>,
 <td><a href="https://www.webometrics.info/en/detalles/covenantuniversity.edu.ng" rel="noopener" target="_blank"></a></td>,
 <td>1169</td>,
 <td>3884</td>,
 <td>1356</td>,
 <td>1797</td>,
 <td>3</td>,
 <td>1805</td>,
 <td><a href="http://www.unn.edu.ng/" rel="noreferrer noopener" target="_blank">University of Nigeria</a></td>,
 <td><a href="https://www.webometrics.info/en/detalles/unn.edu.ng" rel="noopener" target="_blank"></a></td>,
 <td>1311</td>,
 <td>3279</td>,
 <td>1038</td>,
 <td>2243</td>,
 <td>4</td>,
 <td>1984</td>,
 <t

In [23]:
rank = rows[0].text
rank

'1'

In [22]:
world_rank = rows[1].text
world_rank

'1322'

In [21]:
uni = rows[2].text
uni

'University of Ibadan'

In [121]:
web = rows[1].find('a')

web['href']

'https://www.webometrics.info/en/Africa/Nigeria?sort=desc&order=World%20Rank'

In [ ]:
# refs = soup.find_all('a')
# for i in refs:
#     if i.has_attr('href'):
#         print(i['href'])